In [236]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

engine = create_engine('sqlite:///:memory:',
                       echo=True
                       )
Session = sessionmaker(bind=engine)
session = Session()

In [237]:
from datetime import datetime

from sqlalchemy import (Table, Column, Integer, Numeric, String, DateTime,
                        ForeignKey, Boolean)
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, backref

Base = declarative_base()

class Cookie(Base):
    __tablename__ = 'cookies'

    cookie_id = Column(Integer(), primary_key=True)
    cookie_name = Column(String(50), index=True)
    cookie_recipe_url = Column(String(255))
    cookie_sku = Column(String(55))
    quantity = Column(Integer())
    unit_cost = Column(Numeric(12, 2))

    def __repr__(self):
        return "Cookie(cookie_name='{self.cookie_name}', " \
                "cookie_recipe_url='{self.cookie_recipe_url}', " \
                "cookie_sku='{self.cookie_sku}', " \
                "quantity={self.quantity}, " \
                "unit_cost={self.unit_cost})".format(self=self)

class User(Base):
    __tablename__ = 'users'

    user_id = Column(Integer(), primary_key=True)
    username = Column(String(15), nullable=False, unique=True)
    email_address = Column(String(255), nullable=False)
    phone = Column(String(20), nullable=False)
    password = Column(String(25), nullable=False)
    created_on = Column(DateTime(), default=datetime.now)
    updated_on = Column(DateTime(), default=datetime.now)

    def __repr__(self):
        return "User(username='{self.username}', " \
            "email_address='{self.email_address}', " \
            "phone='{self.phone}', " \
            "password='{self.password}')".format(self=self)

class Order(Base):
    __tablename__ = 'orders'
    order_id = Column(Integer(), primary_key=True)
    user_id = Column(Integer(), ForeignKey('users.user_id'))
    shipped = Column(Boolean(), default=False)
    user = relationship("User", backref=backref('order', order_by=order_id))

    def __repr__(self):
        return "Order(user_id={self.user_id}), " \
            "shipped={self.shipped})".format(self=self)

class LineItem(Base):
    __tablename__ = 'line_items'
    line_item_id = Column(Integer(), primary_key=True)
    order_id = Column(Integer(), ForeignKey('orders.order_id'))
    cookie_id = Column(Integer(), ForeignKey('cookies.cookie_id'))
    quantity = Column(Integer())
    extended_cost = Column(Numeric(12, 2))
    order = relationship("Order", backref=backref('line_items',
                        order_by=line_item_id))
    cookie = relationship("Cookie", uselist=False)

    def __repr__(self):
        return "LineItems(order_id={self.order_id}, )" \
            "cookie_id={self.cookie_id}" \
            "quantity={self.quantity}, "\
            "extended_cost={self.extended_cost})".format(self=self)

Base.metadata.create_all(engine)

2024-01-03 09:50:14,982 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-03 09:50:14,983 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("cookies")
2024-01-03 09:50:14,984 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-03 09:50:14,986 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("cookies")
2024-01-03 09:50:14,987 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-03 09:50:14,989 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2024-01-03 09:50:14,990 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-03 09:50:14,992 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("users")
2024-01-03 09:50:14,993 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-03 09:50:14,995 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("orders")
2024-01-03 09:50:14,996 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-03 09:50:14,997 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("orders")
2024-01-03 09:50:15,009 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-

In [238]:
cc_cookie = Cookie(cookie_name='chocolate chip',
                   cookie_recipe_url='http://some.aweso.me/cookie/recipe.html',
                   cookie_sku='CC01',
                   quantity=12,
                   unit_cost=0.50)
print(cc_cookie)
print(cc_cookie.cookie_id)
session.add(cc_cookie)
session.commit()
print(cc_cookie.cookie_id)

Cookie(cookie_name='chocolate chip', cookie_recipe_url='http://some.aweso.me/cookie/recipe.html', cookie_sku='CC01', quantity=12, unit_cost=0.5)
None
2024-01-03 09:50:15,057 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-03 09:50:15,060 INFO sqlalchemy.engine.Engine INSERT INTO cookies (cookie_name, cookie_recipe_url, cookie_sku, quantity, unit_cost) VALUES (?, ?, ?, ?, ?)
2024-01-03 09:50:15,061 INFO sqlalchemy.engine.Engine [generated in 0.00130s] ('chocolate chip', 'http://some.aweso.me/cookie/recipe.html', 'CC01', 12, 0.5)
2024-01-03 09:50:15,063 INFO sqlalchemy.engine.Engine COMMIT
2024-01-03 09:50:15,064 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-03 09:50:15,067 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id AS cookies_cookie_id, cookies.cookie_name AS cookies_cookie_name, cookies.cookie_recipe_url AS cookies_cookie_recipe_url, cookies.cookie_sku AS cookies_cookie_sku, cookies.quantity AS cookies_quantity, cookies.unit_cost AS cookies_unit_cost 
FROM c

C:\Users\WX847\AppData\Local\Temp\ipykernel_74472\2749201275.py:10: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  print(cc_cookie.cookie_id)


In [239]:
dcc = Cookie(cookie_name='dark chocolate chip',
             cookie_recipe_url='http://some.aweso.me/cookie/recipe_dark.html',
             cookie_sku='CC02',
             quantity=1,
             unit_cost=0.75)
mol = Cookie(cookie_name='molasses',
             cookie_recipe_url='http://some.aweso.me/cookie/recipe_molasses.html',
             cookie_sku='MOL01',
             quantity=1,
             unit_cost=0.80)
session.add(dcc)
session.add(mol)
session.flush()
print(dcc.cookie_id)
print(mol.cookie_id)

2024-01-03 09:50:15,096 INFO sqlalchemy.engine.Engine INSERT INTO cookies (cookie_name, cookie_recipe_url, cookie_sku, quantity, unit_cost) VALUES (?, ?, ?, ?, ?)
2024-01-03 09:50:15,097 INFO sqlalchemy.engine.Engine [cached since 0.03719s ago] ('dark chocolate chip', 'http://some.aweso.me/cookie/recipe_dark.html', 'CC02', 1, 0.75)
2024-01-03 09:50:15,099 INFO sqlalchemy.engine.Engine INSERT INTO cookies (cookie_name, cookie_recipe_url, cookie_sku, quantity, unit_cost) VALUES (?, ?, ?, ?, ?)
2024-01-03 09:50:15,100 INFO sqlalchemy.engine.Engine [cached since 0.0401s ago] ('molasses', 'http://some.aweso.me/cookie/recipe_molasses.html', 'MOL01', 1, 0.8)
2
3


In [240]:
c1 = Cookie(cookie_name='peanut butter',
            cookie_recipe_url='http://some.aweso.me/cookie/peanut.html',
            cookie_sku='PB01',
            quantity=24,
            unit_cost=0.25)
c2 = Cookie(cookie_name='oatmeal raisin',
            cookie_recipe_url='http://some.okay.me/cookie/raisin.html',
            cookie_sku='EWW01',
            quantity=100,
            unit_cost=1.00)
session.bulk_save_objects([c1, c2])
session.commit()
print(c1.cookie_id)


2024-01-03 09:50:15,126 INFO sqlalchemy.engine.Engine INSERT INTO cookies (cookie_name, cookie_recipe_url, cookie_sku, quantity, unit_cost) VALUES (?, ?, ?, ?, ?)
2024-01-03 09:50:15,128 INFO sqlalchemy.engine.Engine [generated in 0.00143s] (('peanut butter', 'http://some.aweso.me/cookie/peanut.html', 'PB01', 24, 0.25), ('oatmeal raisin', 'http://some.okay.me/cookie/raisin.html', 'EWW01', 100, 1.0))
2024-01-03 09:50:15,129 INFO sqlalchemy.engine.Engine COMMIT
None


In [241]:
# Example 7-4 Get all the cookies
cookies = session.query(Cookie).all()
print(cookies)

2024-01-03 09:50:15,154 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-03 09:50:15,156 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id AS cookies_cookie_id, cookies.cookie_name AS cookies_cookie_name, cookies.cookie_recipe_url AS cookies_cookie_recipe_url, cookies.cookie_sku AS cookies_cookie_sku, cookies.quantity AS cookies_quantity, cookies.unit_cost AS cookies_unit_cost 
FROM cookies
2024-01-03 09:50:15,158 INFO sqlalchemy.engine.Engine [generated in 0.00137s] ()
[Cookie(cookie_name='chocolate chip', cookie_recipe_url='http://some.aweso.me/cookie/recipe.html', cookie_sku='CC01', quantity=12, unit_cost=0.50), Cookie(cookie_name='dark chocolate chip', cookie_recipe_url='http://some.aweso.me/cookie/recipe_dark.html', cookie_sku='CC02', quantity=1, unit_cost=0.75), Cookie(cookie_name='molasses', cookie_recipe_url='http://some.aweso.me/cookie/recipe_molasses.html', cookie_sku='MOL01', quantity=1, unit_cost=0.80), Cookie(cookie_name='peanut butter', cookie_recipe_url='http:

In [242]:
for cookie in session.query(Cookie):
    print(cookie)

2024-01-03 09:50:15,171 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id AS cookies_cookie_id, cookies.cookie_name AS cookies_cookie_name, cookies.cookie_recipe_url AS cookies_cookie_recipe_url, cookies.cookie_sku AS cookies_cookie_sku, cookies.quantity AS cookies_quantity, cookies.unit_cost AS cookies_unit_cost 
FROM cookies
2024-01-03 09:50:15,173 INFO sqlalchemy.engine.Engine [cached since 0.01615s ago] ()
Cookie(cookie_name='chocolate chip', cookie_recipe_url='http://some.aweso.me/cookie/recipe.html', cookie_sku='CC01', quantity=12, unit_cost=0.50)
Cookie(cookie_name='dark chocolate chip', cookie_recipe_url='http://some.aweso.me/cookie/recipe_dark.html', cookie_sku='CC02', quantity=1, unit_cost=0.75)
Cookie(cookie_name='molasses', cookie_recipe_url='http://some.aweso.me/cookie/recipe_molasses.html', cookie_sku='MOL01', quantity=1, unit_cost=0.80)
Cookie(cookie_name='peanut butter', cookie_recipe_url='http://some.aweso.me/cookie/peanut.html', cookie_sku='PB01', quantity=24, un

In [243]:
# Example 7-6 Select only cookie_name and quantity

print(session.query(Cookie.cookie_name, Cookie.quantity).first())

2024-01-03 09:50:15,189 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_name AS cookies_cookie_name, cookies.quantity AS cookies_quantity 
FROM cookies
 LIMIT ? OFFSET ?
2024-01-03 09:50:15,191 INFO sqlalchemy.engine.Engine [generated in 0.00158s] (1, 0)
('chocolate chip', 12)


In [244]:
for cookie in session.query(Cookie).order_by(Cookie.quantity):
    print('{:3} - {}'.format(cookie.quantity, cookie.cookie_name))

2024-01-03 09:50:15,219 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id AS cookies_cookie_id, cookies.cookie_name AS cookies_cookie_name, cookies.cookie_recipe_url AS cookies_cookie_recipe_url, cookies.cookie_sku AS cookies_cookie_sku, cookies.quantity AS cookies_quantity, cookies.unit_cost AS cookies_unit_cost 
FROM cookies ORDER BY cookies.quantity
2024-01-03 09:50:15,220 INFO sqlalchemy.engine.Engine [generated in 0.00114s] ()
  1 - dark chocolate chip
  1 - molasses
 12 - chocolate chip
 24 - peanut butter
100 - oatmeal raisin


In [245]:
from sqlalchemy import desc
for cookie in session.query(Cookie).order_by(desc(Cookie.quantity)):
    print('{:3} - {}'.format(cookie.quantity, cookie.cookie_name))

2024-01-03 09:50:15,240 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id AS cookies_cookie_id, cookies.cookie_name AS cookies_cookie_name, cookies.cookie_recipe_url AS cookies_cookie_recipe_url, cookies.cookie_sku AS cookies_cookie_sku, cookies.quantity AS cookies_quantity, cookies.unit_cost AS cookies_unit_cost 
FROM cookies ORDER BY cookies.quantity DESC
2024-01-03 09:50:15,242 INFO sqlalchemy.engine.Engine [generated in 0.00147s] ()
100 - oatmeal raisin
 24 - peanut butter
 12 - chocolate chip
  1 - dark chocolate chip
  1 - molasses


In [246]:
# Example 7-9 Two fewest cookie inventories
query = session.query(Cookie).order_by(Cookie.quantity)[:2]
print([result.cookie_name for result in query])

2024-01-03 09:50:15,267 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id AS cookies_cookie_id, cookies.cookie_name AS cookies_cookie_name, cookies.cookie_recipe_url AS cookies_cookie_recipe_url, cookies.cookie_sku AS cookies_cookie_sku, cookies.quantity AS cookies_quantity, cookies.unit_cost AS cookies_unit_cost 
FROM cookies ORDER BY cookies.quantity
 LIMIT ? OFFSET ?
2024-01-03 09:50:15,269 INFO sqlalchemy.engine.Engine [generated in 0.00148s] (2, 0)
['dark chocolate chip', 'molasses']


In [247]:
# Example 7-10 Two fewest cookie inventories with limit
query = session.query(Cookie).order_by(Cookie.quantity).limit(2)
print([result.cookie_name for result in query])

2024-01-03 09:50:15,286 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id AS cookies_cookie_id, cookies.cookie_name AS cookies_cookie_name, cookies.cookie_recipe_url AS cookies_cookie_recipe_url, cookies.cookie_sku AS cookies_cookie_sku, cookies.quantity AS cookies_quantity, cookies.unit_cost AS cookies_unit_cost 
FROM cookies ORDER BY cookies.quantity
 LIMIT ? OFFSET ?
2024-01-03 09:50:15,287 INFO sqlalchemy.engine.Engine [cached since 0.02012s ago] (2, 0)
['dark chocolate chip', 'molasses']


In [248]:
# Exampe 7-11 Summing our cookies
from sqlalchemy import func
inv_count = session.query(func.sum(Cookie.quantity)).scalar()
print(inv_count)

2024-01-03 09:50:15,313 INFO sqlalchemy.engine.Engine SELECT sum(cookies.quantity) AS sum_1 
FROM cookies
2024-01-03 09:50:15,315 INFO sqlalchemy.engine.Engine [generated in 0.00116s] ()
138


In [249]:
rec_count = session.query(func.count(Cookie.cookie_name)).first()
print(rec_count)

2024-01-03 09:50:15,337 INFO sqlalchemy.engine.Engine SELECT count(cookies.cookie_name) AS count_1 
FROM cookies
 LIMIT ? OFFSET ?
2024-01-03 09:50:15,339 INFO sqlalchemy.engine.Engine [generated in 0.00177s] (1, 0)
(5,)


In [250]:
# Example 7-13 Renaming our count column
rec_count = session.query(func.count(Cookie.cookie_name)
                          .label('inventory_count')).first()
print(rec_count.keys())
print(rec_count.inventory_count)


2024-01-03 09:50:15,377 INFO sqlalchemy.engine.Engine SELECT count(cookies.cookie_name) AS inventory_count 
FROM cookies
 LIMIT ? OFFSET ?
2024-01-03 09:50:15,378 INFO sqlalchemy.engine.Engine [generated in 0.00161s] (1, 0)
RMKeyView(['inventory_count'])
5


In [251]:
record = session.query(Cookie).filter(Cookie.cookie_name == 'chocolate chip').first()
print(record)

2024-01-03 09:50:15,410 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id AS cookies_cookie_id, cookies.cookie_name AS cookies_cookie_name, cookies.cookie_recipe_url AS cookies_cookie_recipe_url, cookies.cookie_sku AS cookies_cookie_sku, cookies.quantity AS cookies_quantity, cookies.unit_cost AS cookies_unit_cost 
FROM cookies 
WHERE cookies.cookie_name = ?
 LIMIT ? OFFSET ?
2024-01-03 09:50:15,411 INFO sqlalchemy.engine.Engine [generated in 0.00098s] ('chocolate chip', 1, 0)
Cookie(cookie_name='chocolate chip', cookie_recipe_url='http://some.aweso.me/cookie/recipe.html', cookie_sku='CC01', quantity=12, unit_cost=0.50)


In [252]:
# Example 7-16 Finding names with "chocolate" in them
query = session.query(Cookie).filter(Cookie.cookie_name.like('%chocolate%'))
for record in query:
    print(record.cookie_name)

2024-01-03 09:50:15,576 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id AS cookies_cookie_id, cookies.cookie_name AS cookies_cookie_name, cookies.cookie_recipe_url AS cookies_cookie_recipe_url, cookies.cookie_sku AS cookies_cookie_sku, cookies.quantity AS cookies_quantity, cookies.unit_cost AS cookies_unit_cost 
FROM cookies 
WHERE cookies.cookie_name LIKE ?
2024-01-03 09:50:15,579 INFO sqlalchemy.engine.Engine [generated in 0.00261s] ('%chocolate%',)
chocolate chip
dark chocolate chip


In [253]:
results = session.query(Cookie.cookie_name, 'SKU-' + Cookie.cookie_sku).all()
for row in results:
    print(row)

2024-01-03 09:50:15,638 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_name AS cookies_cookie_name, ? || cookies.cookie_sku AS anon_1 
FROM cookies
2024-01-03 09:50:15,640 INFO sqlalchemy.engine.Engine [generated in 0.00246s] ('SKU-',)
('chocolate chip', 'SKU-CC01')
('dark chocolate chip', 'SKU-CC02')
('molasses', 'SKU-MOL01')
('peanut butter', 'SKU-PB01')
('oatmeal raisin', 'SKU-EWW01')


In [254]:
# Example 7-18 Inventory value by cookie
from sqlalchemy import cast
query = session.query(Cookie.cookie_name,
                      cast((Cookie.quantity * Cookie.unit_cost),
                           Numeric(12, 2)).label('inv_cost'))
for result in query:
    print('{} - {}'.format(result.cookie_name, result.inv_cost))

2024-01-03 09:50:15,671 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_name AS cookies_cookie_name, CAST(cookies.quantity * cookies.unit_cost AS NUMERIC(12, 2)) AS inv_cost 
FROM cookies
2024-01-03 09:50:15,672 INFO sqlalchemy.engine.Engine [generated in 0.00125s] ()
chocolate chip - 6.00
dark chocolate chip - 0.75
molasses - 0.80
peanut butter - 6.00
oatmeal raisin - 100.00


C:\Users\WX847\AppData\Local\Temp\ipykernel_74472\1227448208.py:6: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  for result in query:


In [255]:
# Example 7-18 Inventory value by cookie
from sqlalchemy import cast
query = session.query(Cookie.cookie_name,
                      (Cookie.quantity * Cookie.unit_cost).label('inv_cost'))
for result in query:
    print('{} - {}'.format(result.cookie_name, result.inv_cost))

2024-01-03 09:50:15,711 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_name AS cookies_cookie_name, cookies.quantity * cookies.unit_cost AS inv_cost 
FROM cookies
2024-01-03 09:50:15,712 INFO sqlalchemy.engine.Engine [generated in 0.00181s] ()
chocolate chip - 6.00
dark chocolate chip - 0.75
molasses - 0.80
peanut butter - 6.00
oatmeal raisin - 100.00


In [256]:
# Example 7-19 Using filter with multiple ClauseElement expression to perform an AND
query = session.query(Cookie).filter(
    Cookie.quantity > 23,
    Cookie.unit_cost < 0.40
)
for result in query:
    print(result.cookie_name)

2024-01-03 09:50:15,844 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id AS cookies_cookie_id, cookies.cookie_name AS cookies_cookie_name, cookies.cookie_recipe_url AS cookies_cookie_recipe_url, cookies.cookie_sku AS cookies_cookie_sku, cookies.quantity AS cookies_quantity, cookies.unit_cost AS cookies_unit_cost 
FROM cookies 
WHERE cookies.quantity > ? AND cookies.unit_cost < ?
2024-01-03 09:50:15,846 INFO sqlalchemy.engine.Engine [generated in 0.00130s] (23, 0.4)
peanut butter


In [257]:
# Example 7-20 Using the or() conjunction
from sqlalchemy import and_, or_, not_

query = session.query(Cookie).filter(
    or_(
        Cookie.quantity.between(10, 50),
        Cookie.cookie_name.contains('chip')
    )
)

for result in query:
    print(result.cookie_name)

2024-01-03 09:50:15,870 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id AS cookies_cookie_id, cookies.cookie_name AS cookies_cookie_name, cookies.cookie_recipe_url AS cookies_cookie_recipe_url, cookies.cookie_sku AS cookies_cookie_sku, cookies.quantity AS cookies_quantity, cookies.unit_cost AS cookies_unit_cost 
FROM cookies 
WHERE cookies.quantity BETWEEN ? AND ? OR (cookies.cookie_name LIKE '%' || ? || '%')
2024-01-03 09:50:15,871 INFO sqlalchemy.engine.Engine [generated in 0.00139s] (10, 50, 'chip')
chocolate chip
dark chocolate chip
peanut butter


In [258]:
# Example 7-21 Updating data via object
query = session.query(Cookie)
cc_cookie = query.filter(Cookie.cookie_name=="chocolate chip").first()
cc_cookie.quantity = cc_cookie.quantity + 120
session.commit()
print(cc_cookie.quantity)

2024-01-03 09:50:15,901 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id AS cookies_cookie_id, cookies.cookie_name AS cookies_cookie_name, cookies.cookie_recipe_url AS cookies_cookie_recipe_url, cookies.cookie_sku AS cookies_cookie_sku, cookies.quantity AS cookies_quantity, cookies.unit_cost AS cookies_unit_cost 
FROM cookies 
WHERE cookies.cookie_name = ?
 LIMIT ? OFFSET ?
2024-01-03 09:50:15,902 INFO sqlalchemy.engine.Engine [cached since 0.4919s ago] ('chocolate chip', 1, 0)
2024-01-03 09:50:15,905 INFO sqlalchemy.engine.Engine UPDATE cookies SET quantity=? WHERE cookies.cookie_id = ?
2024-01-03 09:50:15,907 INFO sqlalchemy.engine.Engine [generated in 0.00160s] (132, 1)
2024-01-03 09:50:15,909 INFO sqlalchemy.engine.Engine COMMIT
2024-01-03 09:50:15,912 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-03 09:50:15,913 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id AS cookies_cookie_id, cookies.cookie_name AS cookies_cookie_name, cookies.cookie_recipe_url AS cookie

In [259]:
# Example 7-22 Updating data in place
query = session.query(Cookie)
query = query.filter(Cookie.cookie_name == "chocolate chip")
query.update({Cookie.quantity: Cookie.quantity - 20})

cc_cookie = query.first()
print(cc_cookie.quantity)

2024-01-03 09:50:15,934 INFO sqlalchemy.engine.Engine UPDATE cookies SET quantity=(cookies.quantity - ?) WHERE cookies.cookie_name = ?
2024-01-03 09:50:15,935 INFO sqlalchemy.engine.Engine [generated in 0.00113s] (20, 'chocolate chip')
2024-01-03 09:50:15,938 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id AS cookies_cookie_id, cookies.cookie_name AS cookies_cookie_name, cookies.cookie_recipe_url AS cookies_cookie_recipe_url, cookies.cookie_sku AS cookies_cookie_sku, cookies.quantity AS cookies_quantity, cookies.unit_cost AS cookies_unit_cost 
FROM cookies 
WHERE cookies.cookie_name = ?
 LIMIT ? OFFSET ?
2024-01-03 09:50:15,938 INFO sqlalchemy.engine.Engine [cached since 0.5283s ago] ('chocolate chip', 1, 0)
112


In [260]:
# Example 7-23 Deleting data
query = session.query(Cookie)
query = query.filter(Cookie.cookie_name == "dark chocolate chip")
dcc_cookie = query.one()
session.delete(dcc_cookie)
session.commit()
dcc_cookie = query.first()
print(dcc_cookie)

2024-01-03 09:50:15,964 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id AS cookies_cookie_id, cookies.cookie_name AS cookies_cookie_name, cookies.cookie_recipe_url AS cookies_cookie_recipe_url, cookies.cookie_sku AS cookies_cookie_sku, cookies.quantity AS cookies_quantity, cookies.unit_cost AS cookies_unit_cost 
FROM cookies 
WHERE cookies.cookie_name = ?
2024-01-03 09:50:15,965 INFO sqlalchemy.engine.Engine [generated in 0.00121s] ('dark chocolate chip',)
2024-01-03 09:50:15,968 INFO sqlalchemy.engine.Engine DELETE FROM cookies WHERE cookies.cookie_id = ?
2024-01-03 09:50:15,969 INFO sqlalchemy.engine.Engine [generated in 0.00108s] (2,)
2024-01-03 09:50:15,972 INFO sqlalchemy.engine.Engine COMMIT
2024-01-03 09:50:15,976 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-03 09:50:15,978 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id AS cookies_cookie_id, cookies.cookie_name AS cookies_cookie_name, cookies.cookie_recipe_url AS cookies_cookie_recipe_url, cookies.cookie

In [261]:
# Example 7-24 Deleting data
query = session.query(Cookie)
query = query.filter(Cookie.cookie_name=="molasses")
query.delete()
mol_cookie = query.first()
print(mol_cookie)

2024-01-03 09:50:15,997 INFO sqlalchemy.engine.Engine DELETE FROM cookies WHERE cookies.cookie_name = ?
2024-01-03 09:50:15,998 INFO sqlalchemy.engine.Engine [generated in 0.00161s] ('molasses',)
2024-01-03 09:50:16,000 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id AS cookies_cookie_id, cookies.cookie_name AS cookies_cookie_name, cookies.cookie_recipe_url AS cookies_cookie_recipe_url, cookies.cookie_sku AS cookies_cookie_sku, cookies.quantity AS cookies_quantity, cookies.unit_cost AS cookies_unit_cost 
FROM cookies 
WHERE cookies.cookie_name = ?
 LIMIT ? OFFSET ?
2024-01-03 09:50:16,001 INFO sqlalchemy.engine.Engine [cached since 0.5908s ago] ('molasses', 1, 0)
None


In [262]:
cookiemon = User(username='cookiemon',
                 email_address='mon@cookie.com',
                 phone='111-111-111',
                 password='password')
cakeeater = User(username='cakeeater',
                 email_address='cakeeater@cake.com',
                 phone='password',
                 password='password')
pieperson = User(username='pieperson',
                 email_address='person@pie.com',
                 phone='333-333-3333',
                 password='password')
session.add(cookiemon)
session.add(cakeeater)
session.add(pieperson)
session.commit()

2024-01-03 09:50:16,031 INFO sqlalchemy.engine.Engine INSERT INTO users (username, email_address, phone, password, created_on, updated_on) VALUES (?, ?, ?, ?, ?, ?)
2024-01-03 09:50:16,034 INFO sqlalchemy.engine.Engine [generated in 0.00325s] ('cookiemon', 'mon@cookie.com', '111-111-111', 'password', '2024-01-03 09:50:16.031867', '2024-01-03 09:50:16.031867')
2024-01-03 09:50:16,036 INFO sqlalchemy.engine.Engine INSERT INTO users (username, email_address, phone, password, created_on, updated_on) VALUES (?, ?, ?, ?, ?, ?)
2024-01-03 09:50:16,037 INFO sqlalchemy.engine.Engine [cached since 0.006614s ago] ('cakeeater', 'cakeeater@cake.com', 'password', 'password', '2024-01-03 09:50:16.036868', '2024-01-03 09:50:16.036868')
2024-01-03 09:50:16,039 INFO sqlalchemy.engine.Engine INSERT INTO users (username, email_address, phone, password, created_on, updated_on) VALUES (?, ?, ?, ?, ?, ?)
2024-01-03 09:50:16,041 INFO sqlalchemy.engine.Engine [cached since 0.01023s ago] ('pieperson', 'person@p

In [263]:
o1 = Order()  # 创建一个订单
o1.user = cookiemon  # 添加订单的用户
session.add(o1)
# 订单内的第一个产品线
cc = session.query(Cookie).filter(Cookie.cookie_name=='chocolate chip').one()
line1 = LineItem(cookie=cc, quantity=2, extended_cost=1.00)
# 订单内的第二个产品线
pb = session.query(Cookie).filter(Cookie.cookie_name=='peanut butter').one()
line2 = LineItem(quantity=12, extended_cost=3.00)
line2.cookie = pb
line2.order = o1

o1.line_items.append(line1)
o1.line_items.append(line2)
session.commit()

2024-01-03 09:50:16,062 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-03 09:50:16,065 INFO sqlalchemy.engine.Engine SELECT users.user_id AS users_user_id, users.username AS users_username, users.email_address AS users_email_address, users.phone AS users_phone, users.password AS users_password, users.created_on AS users_created_on, users.updated_on AS users_updated_on 
FROM users 
WHERE users.user_id = ?
2024-01-03 09:50:16,067 INFO sqlalchemy.engine.Engine [generated in 0.00158s] (1,)
2024-01-03 09:50:16,070 INFO sqlalchemy.engine.Engine INSERT INTO orders (user_id, shipped) VALUES (?, ?)
2024-01-03 09:50:16,071 INFO sqlalchemy.engine.Engine [generated in 0.00112s] (1, 0)
2024-01-03 09:50:16,074 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id AS cookies_cookie_id, cookies.cookie_name AS cookies_cookie_name, cookies.cookie_recipe_url AS cookies_cookie_recipe_url, cookies.cookie_sku AS cookies_cookie_sku, cookies.quantity AS cookies_quantity, cookies.unit_cost AS cookies_

C:\Users\WX847\AppData\Local\Temp\ipykernel_74472\4294402812.py:13: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  o1.line_items.append(line1)


In [264]:
o2 = Order()
o2.user = cakeeater

cc = session.query(Cookie).filter(Cookie.cookie_name == 'chocolate chip').one()
line1 = LineItem(cookie=cc, quantity=25, extended_cost=12.00)
oat = session.query(Cookie).filter(Cookie.cookie_name == 'oatmeal raisin').one()
line2 = LineItem(cookie=oat, quantity=6, extended_cost=6.00)

o2.line_items.append(line1)
o2.line_items.append(line2)

session.add(o2)
session.commit()

2024-01-03 09:50:16,113 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-03 09:50:16,115 INFO sqlalchemy.engine.Engine SELECT users.user_id AS users_user_id, users.username AS users_username, users.email_address AS users_email_address, users.phone AS users_phone, users.password AS users_password, users.created_on AS users_created_on, users.updated_on AS users_updated_on 
FROM users 
WHERE users.user_id = ?
2024-01-03 09:50:16,117 INFO sqlalchemy.engine.Engine [cached since 0.05165s ago] (2,)
2024-01-03 09:50:16,120 INFO sqlalchemy.engine.Engine INSERT INTO orders (user_id, shipped) VALUES (?, ?)
2024-01-03 09:50:16,121 INFO sqlalchemy.engine.Engine [cached since 0.05139s ago] (2, 0)
2024-01-03 09:50:16,124 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id AS cookies_cookie_id, cookies.cookie_name AS cookies_cookie_name, cookies.cookie_recipe_url AS cookies_cookie_recipe_url, cookies.cookie_sku AS cookies_cookie_sku, cookies.quantity AS cookies_quantity, cookies.unit_cost AS 

In [265]:
# Example 7-26 Using join to select from multiple tables
query = session.query(Order.order_id, User.username, User.phone,
                      Cookie.cookie_name, LineItem.quantity,
                      LineItem.extended_cost)
query = query.join(User).join(LineItem).join(Cookie)
results = query.filter(User.username == 'cookiemon').all()
print(results)

2024-01-03 09:50:16,157 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-03 09:50:16,162 INFO sqlalchemy.engine.Engine SELECT orders.order_id AS orders_order_id, users.username AS users_username, users.phone AS users_phone, cookies.cookie_name AS cookies_cookie_name, line_items.quantity AS line_items_quantity, line_items.extended_cost AS line_items_extended_cost 
FROM orders JOIN users ON users.user_id = orders.user_id JOIN line_items ON orders.order_id = line_items.order_id JOIN cookies ON cookies.cookie_id = line_items.cookie_id 
WHERE users.username = ?
2024-01-03 09:50:16,164 INFO sqlalchemy.engine.Engine [generated in 0.00167s] ('cookiemon',)
[(1, 'cookiemon', '111-111-111', 'peanut butter', 12, Decimal('3.00')), (1, 'cookiemon', '111-111-111', 'chocolate chip', 2, Decimal('1.00'))]


In [266]:
# Example 7-27 Using outerjoin to select from multiple tables
query = session.query(User.username, func.count(Order.order_id))
query = query.outerjoin(Order).group_by(User.username)
for row in query:
    print(row)

2024-01-03 09:50:16,187 INFO sqlalchemy.engine.Engine SELECT users.username AS users_username, count(orders.order_id) AS count_1 
FROM users LEFT OUTER JOIN orders ON users.user_id = orders.user_id GROUP BY users.username
2024-01-03 09:50:16,188 INFO sqlalchemy.engine.Engine [generated in 0.00145s] ()
('cakeeater', 1)
('cookiemon', 1)
('pieperson', 0)


In [267]:
class Employee(Base):
    __tablename__ = 'employees'

    id = Column(Integer(), primary_key=True)
    manager_id = Column(Integer(), ForeignKey('employees.id'))
    name = Column(String(255), nullable=False)

    manage = relationship("Employee", backref=backref('reports'),
                          remote_side=[id])
Base.metadata.create_all(engine)

2024-01-03 09:50:16,223 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-03 09:50:16,225 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("cookies")
2024-01-03 09:50:16,226 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-03 09:50:16,228 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2024-01-03 09:50:16,228 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-03 09:50:16,229 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("orders")
2024-01-03 09:50:16,230 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-03 09:50:16,231 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("line_items")
2024-01-03 09:50:16,231 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-03 09:50:16,233 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("employees")
2024-01-03 09:50:16,233 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-03 09:50:16,235 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("employees")
2024-01-03 09:50:16,235 INFO sqlalchemy.engine.Engine [raw sql

In [268]:
marsha = Employee(name='Marsha')
fred = Employee(name='Fred')

marsha.reports.append(fred)

session.add(marsha)
session.commit()

2024-01-03 09:50:16,270 INFO sqlalchemy.engine.Engine INSERT INTO employees (manager_id, name) VALUES (?, ?)
2024-01-03 09:50:16,272 INFO sqlalchemy.engine.Engine [generated in 0.00161s] (None, 'Marsha')
2024-01-03 09:50:16,276 INFO sqlalchemy.engine.Engine INSERT INTO employees (manager_id, name) VALUES (?, ?)
2024-01-03 09:50:16,277 INFO sqlalchemy.engine.Engine [cached since 0.006449s ago] (1, 'Fred')
2024-01-03 09:50:16,278 INFO sqlalchemy.engine.Engine COMMIT


In [269]:
for report in marsha.reports:
    print(report.name)

2024-01-03 09:50:16,345 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-03 09:50:16,349 INFO sqlalchemy.engine.Engine SELECT employees.id AS employees_id, employees.manager_id AS employees_manager_id, employees.name AS employees_name 
FROM employees 
WHERE employees.id = ?
2024-01-03 09:50:16,350 INFO sqlalchemy.engine.Engine [generated in 0.00136s] (1,)
2024-01-03 09:50:16,353 INFO sqlalchemy.engine.Engine SELECT employees.id AS employees_id, employees.manager_id AS employees_manager_id, employees.name AS employees_name 
FROM employees 
WHERE ? = employees.manager_id
2024-01-03 09:50:16,354 INFO sqlalchemy.engine.Engine [generated in 0.00113s] (1,)
Fred


In [270]:
# Example 7-28 Grouping data
query = session.query(User.username, func.count(Order.order_id))
query = query.outerjoin(Order).group_by(User.username)
for row in query:
    print(row)

2024-01-03 09:50:16,377 INFO sqlalchemy.engine.Engine SELECT users.username AS users_username, count(orders.order_id) AS count_1 
FROM users LEFT OUTER JOIN orders ON users.user_id = orders.user_id GROUP BY users.username
2024-01-03 09:50:16,396 INFO sqlalchemy.engine.Engine [cached since 0.2091s ago] ()
('cakeeater', 1)
('cookiemon', 1)
('pieperson', 0)


In [271]:
# Example 7-29 Chaining
def get_orders_by_customer(cust_name):
    query = session.query(Order.order_id, User.username, User.phone,
                          Cookie.cookie_name, LineItem.quantity,
                          LineItem.extended_cost)
    query = query.join(User).join(LineItem).join(Cookie)
    results = query.filter(User.username == cust_name).all()
    return results

get_orders_by_customer('cakeeater')

2024-01-03 09:50:16,424 INFO sqlalchemy.engine.Engine SELECT orders.order_id AS orders_order_id, users.username AS users_username, users.phone AS users_phone, cookies.cookie_name AS cookies_cookie_name, line_items.quantity AS line_items_quantity, line_items.extended_cost AS line_items_extended_cost 
FROM orders JOIN users ON users.user_id = orders.user_id JOIN line_items ON orders.order_id = line_items.order_id JOIN cookies ON cookies.cookie_id = line_items.cookie_id 
WHERE users.username = ?
2024-01-03 09:50:16,425 INFO sqlalchemy.engine.Engine [cached since 0.2633s ago] ('cakeeater',)


[(2, 'cakeeater', 'password', 'chocolate chip', 25, Decimal('12.00')),
 (2, 'cakeeater', 'password', 'oatmeal raisin', 6, Decimal('6.00'))]

In [273]:
# Example 7-30 Conditional chaining
def get_orders_by_customer(cust_name, shipped=None, details=False):
    query = session.query(Order.order_id, User.username, User.phone)
    query = query.join(User)
    if details:
        query = query.add_columns(Cookie.cookie_name, LineItem.quantity,
                                  LineItem.extended_cost)
        query = query.join(LineItem).join(Cookie)
    if shipped is not None:
        query = query.where(Order.shipped == shipped)
    results = query.filter(User.username == cust_name).all()
    return results

get_orders_by_customer('cakeeater')


2024-01-03 09:51:09,852 INFO sqlalchemy.engine.Engine SELECT orders.order_id AS orders_order_id, users.username AS users_username, users.phone AS users_phone 
FROM orders JOIN users ON users.user_id = orders.user_id 
WHERE users.username = ?
2024-01-03 09:51:09,854 INFO sqlalchemy.engine.Engine [cached since 53.4s ago] ('cakeeater',)


[(2, 'cakeeater', 'password')]

In [274]:
get_orders_by_customer('cakeeater', details=True)


2024-01-03 09:51:14,592 INFO sqlalchemy.engine.Engine SELECT orders.order_id AS orders_order_id, users.username AS users_username, users.phone AS users_phone, cookies.cookie_name AS cookies_cookie_name, line_items.quantity AS line_items_quantity, line_items.extended_cost AS line_items_extended_cost 
FROM orders JOIN users ON users.user_id = orders.user_id JOIN line_items ON orders.order_id = line_items.order_id JOIN cookies ON cookies.cookie_id = line_items.cookie_id 
WHERE users.username = ?
2024-01-03 09:51:14,594 INFO sqlalchemy.engine.Engine [cached since 58.43s ago] ('cakeeater',)


[(2, 'cakeeater', 'password', 'chocolate chip', 25, Decimal('12.00')),
 (2, 'cakeeater', 'password', 'oatmeal raisin', 6, Decimal('6.00'))]

In [275]:
get_orders_by_customer('cakeeater', shipped=True)


2024-01-03 09:51:20,735 INFO sqlalchemy.engine.Engine SELECT orders.order_id AS orders_order_id, users.username AS users_username, users.phone AS users_phone 
FROM orders JOIN users ON users.user_id = orders.user_id 
WHERE orders.shipped = 1 AND users.username = ?
2024-01-03 09:51:20,737 INFO sqlalchemy.engine.Engine [cached since 64.28s ago] ('cakeeater',)


[]

In [276]:
get_orders_by_customer('cakeeater', shipped=False)


2024-01-03 09:51:27,071 INFO sqlalchemy.engine.Engine SELECT orders.order_id AS orders_order_id, users.username AS users_username, users.phone AS users_phone 
FROM orders JOIN users ON users.user_id = orders.user_id 
WHERE orders.shipped = 0 AND users.username = ?
2024-01-03 09:51:27,074 INFO sqlalchemy.engine.Engine [cached since 70.61s ago] ('cakeeater',)


[(2, 'cakeeater', 'password')]

In [277]:
get_orders_by_customer('cakeeater', shipped=False, details=True)

2024-01-03 09:51:31,904 INFO sqlalchemy.engine.Engine SELECT orders.order_id AS orders_order_id, users.username AS users_username, users.phone AS users_phone, cookies.cookie_name AS cookies_cookie_name, line_items.quantity AS line_items_quantity, line_items.extended_cost AS line_items_extended_cost 
FROM orders JOIN users ON users.user_id = orders.user_id JOIN line_items ON orders.order_id = line_items.order_id JOIN cookies ON cookies.cookie_id = line_items.cookie_id 
WHERE orders.shipped = 0 AND users.username = ?
2024-01-03 09:51:31,907 INFO sqlalchemy.engine.Engine [cached since 75.44s ago] ('cakeeater',)


[(2, 'cakeeater', 'password', 'chocolate chip', 25, Decimal('12.00')),
 (2, 'cakeeater', 'password', 'oatmeal raisin', 6, Decimal('6.00'))]

In [278]:
# Example 7-31 Partial text query
from sqlalchemy import text
query = session.query(User).filter(text("username = 'cookiemon'"))
print(query.all())

2024-01-03 09:53:50,447 INFO sqlalchemy.engine.Engine SELECT users.user_id AS users_user_id, users.username AS users_username, users.email_address AS users_email_address, users.phone AS users_phone, users.password AS users_password, users.created_on AS users_created_on, users.updated_on AS users_updated_on 
FROM users 
WHERE username = 'cookiemon'
2024-01-03 09:53:50,449 INFO sqlalchemy.engine.Engine [generated in 0.00196s] ()
[User(username='cookiemon', email_address='mon@cookie.com', phone='111-111-111', password='password')]
